<a href="https://colab.research.google.com/github/IlyaZutler/Bus_lanes/blob/main/DM%20_%20LinesGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import LineString, Point
from shapely.ops import unary_union
from shapely.strtree import STRtree
import folium
from geopy.distance import geodesic

In [82]:
# Загрузка данных о Выделенных полосах
gdf_bus_lanes = gpd.read_file("bus-lanes.geojson")
# Выделяем линии Москвы
gdf_bus_lanes = gdf_bus_lanes[gdf_bus_lanes['short_name'] == 'Москва']
# Устанавливаем корректную CRS
gdf_bus_lanes = gdf_bus_lanes.set_crs("EPSG:4326")
# gdf_bus_lanes.sample(1)

In [83]:
# Преобразуем CRS в метры (EPSG:3857)
gdf_bus_lanes = gdf_bus_lanes.to_crs(epsg=3857)
# Делаем buffer на 30 метров
gdf_bus_lanes["geometry"] = gdf_bus_lanes.geometry.buffer(30)
# Возвращаем обратно в географические координаты (EPSG:4326)
gdf_bus_lanes = gdf_bus_lanes.to_crs(epsg=4326)
# gdf_bus_lanes.sample(2)
# Объединяем все POLYGON в один
bus_lanes_union = unary_union(gdf_bus_lanes.geometry)

In [84]:
# 'trips.xlsx' is uploaded to Colab environment
df_trips = pd.read_excel('77601_21.02.25-22.03.25.xlsx', header=0)  # header=0 means the first row is the header

In [85]:
# Split the 'Coordinates' column into 'Longitude' and 'Latitude'
df_trips = df_trips[~pd.isna(df_trips['Сoordinates'])]
df_trips[['Latitude', 'Longitude']] = df_trips['Сoordinates'].str.replace('°', '').str.replace(',', '.').str.split('. ', expand=True)

# Convert 'Day_time' to datetime objects
df_trips['Day_time'] = pd.to_datetime(df_trips['Day_time'], format='%d.%m.%Y %H:%M:%S', errors='coerce')

# df_trips['Ignition'] = df_trips['Ignition'].str.replace('-', '0').str.replace('+', '1')

df_trips = df_trips.sort_values(['Car_ID', 'Day_time'])  # Сортируем по времени

# Преобразование поездок в геометрию
df_trips['geometry'] = [Point(lon, lat) for lon, lat in zip(df_trips['Longitude'], df_trips['Latitude'])]
df_trips = gpd.GeoDataFrame(df_trips, geometry='geometry', crs="EPSG:4326")

# df_trips.sample(1)

In [ ]:
# # вариант 1
# # Проверяем пересечение точек с объединенной зоной
# df_trips["on_bus_lane"] = df_trips["geometry"].apply(lambda point: bus_lanes_union.intersects(point))

In [86]:
# Вариант 2 векторной проверки
# Векторизованная проверка пересечений
df_trips["on_bus_lane"] = gpd.GeoSeries(df_trips["geometry"]).intersects(bus_lanes_union)

In [65]:
# # Вариант 3 с индексацией точек в STRtree
# points = df_trips["geometry"].tolist()  # Список точек shapely.Point

# # Создаем STRtree для точек
# tree = STRtree(points)

# # Запрос: найти точки, которые потенциально пересекаются с полигоном
# candidate_indices = tree.query(bus_lanes_union)  # Индексы точек, чьи bounding box пересекаются

# # Проверяем точное пересечение только для кандидатов
# candidates = [points[i] for i in candidate_indices]
# intersects = [bus_lanes_union.intersects(point) for point in candidates]

# # Создаем результат для всех точек
# df_trips["on_bus_lane"] = False
# for idx, intersect in zip(candidate_indices, intersects):
#     df_trips.loc[idx, "on_bus_lane"] = intersect

In [87]:
# Анализ скорости на участках
df_trips["next_time"] = df_trips.groupby('Car_ID')['Day_time'].shift(-1)
df_trips["next_lon"] = df_trips.groupby('Car_ID')['Longitude'].shift(-1)
df_trips["next_lat"] = df_trips.groupby('Car_ID')['Latitude'].shift(-1)
df_trips["next_on_bus_lane"] = df_trips.groupby('Car_ID')['on_bus_lane'].shift(-1)

# Вычисляем расстояние между последовательными точками в км
# Создаем два массива для координат
coords1 = np.column_stack((df_trips['Latitude'], df_trips['Longitude']))
coords2 = np.column_stack((df_trips['next_lat'], df_trips['next_lon']))

# Функция для вычисления расстояний
def calculate_distances(coords1, coords2):
    return np.array([geodesic((lat1, lon1), (lat2, lon2)).meters / 1000
                     if not pd.isna(lat2) else np.nan
                     for (lat1, lon1), (lat2, lon2) in zip(coords1, coords2)])

# Применяем функцию для всей таблицы
df_trips['distance'] = calculate_distances(coords1, coords2)

# Вычисляем время между последовательными точками, часов, Заменяем NaN на None
time_diff = (df_trips["next_time"] - df_trips['Day_time']).dt.total_seconds() / 3600
df_trips["time"] = time_diff.where(df_trips["next_time"].notna(), None)

# Проверяем, чтобы значения в столбцах "distance" и "time" были валидными, и вычисляем скорость
df_trips["speed"] = np.where(
    (pd.notnull(df_trips["distance"])) & (df_trips["time"].notnull()) & (df_trips["time"] != 0),
    df_trips["distance"] / df_trips["time"],
    None
)

In [88]:
# Оценка скорости на выделенных полосах
avg_speed_bus_lane = 50

# Рассчитываем предсказанное время для каждой строки
df_trips['predicted_time'] = np.where(
    (df_trips["on_bus_lane"] & df_trips["next_on_bus_lane"]),
    np.minimum(df_trips['time'], df_trips['distance'] / avg_speed_bus_lane),
    df_trips['time']
)

In [89]:
# время в движении, часов
total_time = df_trips.loc[df_trips['speed'] > 0, 'time'].sum()
# время в движении с использованием выделенных полос, часов
total_predicted_time = df_trips.loc[df_trips['speed'] > 0, 'predicted_time'].sum()
total_time_saved = total_time - total_predicted_time

print(f"Общее время в движении: {total_time :.2f} часов")
print(f"Общая экономия времени: {total_time_saved :.2f} часов")
print(f"% экономии времени: {total_time_saved / total_time  * 100  :.2f} %")

Общее время в движении: 161.41 часов
Общая экономия времени: 6.20 часов
% экономии времени: 3.84 %
